In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib as mpl

from FinRockConvertCSV import DataReady
from FinRockDataFeeder import PdDataFeeder
from FinRockTradingEnv import TradingEnv
from FinRockEnv import PygameRender
from FinRockMinMax import MinMaxScaler
from FinRockObservations import simpleReward

In [ ]:
df = pd.read_csv("/home/zymantas/Desktop/Training_data/5minData/ATOMUSDT5min.csv")

DataReady(df)

pd_data_feeder  = PdDataFeeder(df)

In [ ]:
env = TradingEnv(
    data_feeder = pd_data_feeder,
    output_transformer = MinMaxScaler(min = pd_data_feeder.min, max = pd_data_feeder.max),
    initial_balance = 1000.0,
    max_episode_steps = 1000,
    window_size = 30,
    reward_function = simpleReward
)

action_space = env.action_space

pygameRender = PygameRender(frame_rate = 8)

In [ ]:
state, info = env.reset()
pygameRender.render(info)
rewards = 0.0

while True:
    # Simulate model prediction, now use random action
    action = np.random.randint(0, action_space)
    # 0 action always hold

    state, reward, terminated, truncated, info = env.step(action)
    rewards += reward
    pygameRender.render(info)

    if terminated or truncated:
        print(info['states'][-1].account_value, rewards)
        rewards = 0.0
        state, info = env.reset()
        pygameRender.reset()